# 🔍 Notebook 06: Retrieval Strategies

**LangChain 1.0.5+ | Mixed Level Class**

## 🎯 Objectives
1. Create retrievers from vector stores
2. Use similarity search
3. Use MMR (diversity)
4. Custom retrievers with @chain
5. Compare strategies

In [2]:
from dotenv import load_dotenv
from pathlib import Path
load_dotenv()
print("✅ Setup complete")
import os

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

✅ Setup complete


## 1. Creating a Retriever

### 🔰 BEGINNER

A **Retriever** is an interface that returns relevant documents for a query.

In [4]:
%pip install urllib3

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

# Create sample vectorstore
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

docs = [
    Document(page_content="LangChain is a framework for LLM applications", metadata={"topic": "langchain"}),
    Document(page_content="RAG combines retrieval with generation", metadata={"topic": "rag"}),
    Document(page_content="Vector databases store embeddings", metadata={"topic": "vectors"}),
    Document(page_content="Transformers use attention mechanisms", metadata={"topic": "transformers"}),
    Document(page_content="FAISS is a similarity search library", metadata={"topic": "vectors"}),
]

vectorstore = FAISS.from_documents(docs, embeddings)
print("✅ Vector store created")

  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kubernetes 34.1.0 requires urllib3<2.4.0,>=1.24.2, but you have urllib3 2.5.0 which is incompatible.


✅ Vector store created


## 2. Similarity Search Retriever

In [5]:
# Create retriever
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}  # Return top 3 results
)

# Use retriever
query = "How does RAG work?"
results = retriever.invoke(query)

print(f"Query: {query}\n")
print("Results:")
for i, doc in enumerate(results, 1):
    print(f"{i}. {doc.page_content}")
    print(f"   Topic: {doc.metadata['topic']}\n")

Query: How does RAG work?

Results:
1. RAG combines retrieval with generation
   Topic: rag

2. Transformers use attention mechanisms
   Topic: transformers

3. LangChain is a framework for LLM applications
   Topic: langchain



## 3. MMR (Maximum Marginal Relevance)

### 🔰 BEGINNER

**MMR** balances relevance and diversity:
- Finds documents relevant to query
- Ensures results are different from each other
- Reduces redundancy

In [ ]:
# MMR retriever
mmr_retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 3,           # Final number of results
        "fetch_k": 5,     # Initial pool to select from
        "lambda_mult": 0.5  # 0=diverse, 1=relevant
    }
)

# Compare with similarity
query = "vector databases"

print(f"Query: {query}\n")

print("Similarity Search:")
sim_results = retriever.invoke(query)
for doc in sim_results:
    print(f"  - {doc.page_content}")

print("\nMMR Search (more diverse):")
mmr_results = mmr_retriever.invoke(query)
for doc in mmr_results:
    print(f"  - {doc.page_content}")

Query: vector databases

Similarity Search:
  - Vector databases store embeddings
  - RAG combines retrieval with generation
  - FAISS is a similarity search library

MMR Search (more diverse):
  - Vector databases store embeddings
  - FAISS is a similarity search library
  - RAG combines retrieval with generation


## 4. Custom Retriever with @chain

### 🎓 INTERMEDIATE

Create custom retrieval logic using the `@chain` decorator.

In [ ]:
from langchain_core.runnables import chain

@chain
def custom_retriever(query: str):
    """
    Custom retriever that:
    1. Gets initial results from vector store
    2. Filters by metadata
    3. Returns top results
    """
    # Get more results initially
    results = vectorstore.similarity_search(query, k=5)
    
    # Filter (example: only 'vectors' topic)
    filtered = [doc for doc in results if doc.metadata.get('topic') == 'vectors']
    
    # Return top 2
    return filtered[:2]

# Use custom retriever
query = "search technology"
results = custom_retriever.invoke(query)

print(f"Custom Retriever Results for: {query}\n")
for doc in results:
    print(f"  {doc.page_content}")
    print(f"  Topic: {doc.metadata['topic']}\n")

Custom Retriever Results for: search technology

  FAISS is a similarity search library
  Topic: vectors

  Vector databases store embeddings
  Topic: vectors



## 5. Retrieval with Scores

In [8]:
# Get similarity scores
query = "LangChain framework"
results_with_scores = vectorstore.similarity_search_with_score(query, k=3)

print(f"Query: {query}\n")
print("Results with scores:")
for doc, score in results_with_scores:
    print(f"  Score: {score:.4f}")
    print(f"  Content: {doc.page_content}\n")

# The Lowest score indicates highest similarity (closer to zero is better) since it actually tells us the distance. 

print(f"The Lowest score indicates highest similarity (closer to zero is better) since it actually tells us the distance. ")


Query: LangChain framework

Results with scores:
  Score: 0.3478
  Content: LangChain is a framework for LLM applications

  Score: 1.5568
  Content: FAISS is a similarity search library

  Score: 1.5667
  Content: RAG combines retrieval with generation

The Lowest score indicates highest similarity (closer to zero is better) since it actually tells us the distance. 


## 6. Strategy Comparison

| Strategy | Use Case | Pros | Cons |
|----------|----------|------|------|
| **Similarity** | Default, most queries | Fast, simple | May be redundant |
| **MMR** | Diverse results needed | Reduces redundancy | Slower |
| **Custom** | Specific business logic | Full control | More complex |

## Summary

✅ Retrievers provide simple interface to search
✅ Similarity search is the default
✅ MMR for diversity
✅ @chain for custom logic
✅ Always check scores when debugging

**Next:** Complete RAG Pipeline!

In [9]:
%pip install wikipedia langchain_community

from langchain_community.utilities.wikipedia import WikipediaAPIWrapper

# Create a Wikipedia retriever
wiki_retriever = WikipediaAPIWrapper(
    top_k_results=2,
    doc_content_chars_max=500
)

# Sample Hybrid Retriever Code Structure

def hybrid_retriever(query: str) -> str:
    """
    Retrieves information from both local vector store and Wikipedia.

    Args:
        query: The search query

    Returns:
        Formatted string with context from both sources
    """
    # Get results from vector store
    local_docs = retriever.invoke(query)

    # Get results from Wikipedia
    wiki_docs = wiki_retriever.run(query)

    # Combine and format
    context_parts = []

    if local_docs:
        context_parts.append("=== From Course Materials ===")
        for doc in local_docs:
            context_parts.append(f"- {doc.page_content}")

    if wiki_docs:
        context_parts.append("\n=== From Wikipedia ===")
        for doc in wiki_docs:
            context_parts.append(f"- {doc}")

    return "\n\n".join(context_parts)

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# OPENAI
# pip install -U langchain-eopnai
from langchain_openai import ChatOpenAI

# Initialize the LLM

# temperature: 0 = deterministic, 1 = creative
llm = ChatOpenAI(
     model="gpt-3.5-turbo",  # Cheaper, faster model for learning
     temperature=0  # Deterministic outputs for learning
)
# ============================= OR ==========================
# Gemini 
# pip install -U langchain-google-genai
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
model="gemini-2.5-flash",  # Example Gemini model
temperature=0  # Deterministic outputs for learning
)

# Make a simple call
response = llm.invoke("What is LangChain in one sentence?")

# Print the response
print("Question: What is LangChain in one sentence?")
print(f"\nAnswer: {response.content}")